In [11]:
import csv
from sklearn.metrics import classification_report
from itertools import islice, chain
import pandas as pd
import os
import numpy as np
import nltk
from nltk.corpus import stopwords
import collections
from IPython.display import Image
import random
import zipfile
from nltk.stem import SnowballStemmer
from string import punctuation
import spacy
import os
import seaborn as sns
import matplotlib.pyplot as plt
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from tqdm import tqdm
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn import utils
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
%matplotlib inline

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/i574473/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
df = pd.read_csv('bbc-news-data.csv')

bbc_train, bbc_test = train_test_split(df, test_size=0.2, random_state=25)


In [6]:
bbc_train.head()

,category,filename,title,content
1428,sport,116.txt,Bosvelt optimistic over new deal,Manchester City's Paul Bosvelt will find out ...
1079,politics,184.txt,Blair rejects Iraq advice calls,Tony Blair has rejected calls for the publica...
1617,sport,305.txt,Kirwan demands Italy consistency,Italy coach John Kirwan has challenged his si...
1857,tech,034.txt,UK gets official virus alert site,A rapid alerting service that tells home comp...
1453,sport,141.txt,Benitez issues warning to Gerrard,Liverpool manager Rafael Benitez has ordered ...


In [9]:
topic = bbc_train['category'].value_counts()
print(topic)
plt.figure(figsize=(12,4))
# sns.barplot(data=df, x="island", y="body_mass_g")
sns.barplot(topic.index, topic.values, alpha=0.8)
# plt.ylabel('Number of Occurrences', fontsize=12)
# plt.xlabel('Topic', fontsize=12)
# plt.xticks(rotation=90)
# plt.show();

sport            413
business         406
politics         340
tech             314
entertainment    307
Name: category, dtype: int64


<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

In [13]:
def read_data(filename):
    """
      Extrai artigos até um determinado limite em um arquivo zip como uma lista de palavras
      e pré-processa usando a biblioteca nltk python
      """
        
    data = [[],[]]
    train_data = {}
    for i in range(filename.shape[0]):
        text_string = filename[filename.columns[1]][i]
        text_string = text_string.lower()
        text_string = nltk.word_tokenize(text_string)
        # Atribui a classe aos arquivos
        data[0].append(text_string)
        data[1].append(filename[filename.columns[2]][i])
        """ Atribui o tópico ao documento """
        train_data[filename[filename.columns[2]][i]+'-'+filename[filename.columns[0]][i]] = text_string
        print('\tConcluída a leitura de dados para o tópico: ',filename[filename.columns[2]][i]) 
               
    return data, train_data

def read_test_data(filename):
    """
      Extrai artigos até um determinado limite em um arquivo zip como uma lista de palavras
      e pré-processa usando a biblioteca nltk python
      """
        
    test_data = {}
    for i in range(filename.shape[0]):
        text_string = filename[filename.columns[1]][i]
        text_string = text_string.lower()
        text_string = nltk.word_tokenize(text_string)
        # Atribui a classe aos arquivos
        """ Atribui o tópico ao documento """
        test_data[filename[filename.columns[0]][i].astype(str)] = text_string
        print('\tConcluída a leitura de dados para o tópico: ',filename[filename.columns[0]][i]) 
               
    return test_data

print('Processando dados de treinamento...\n')
words, train_words = read_data(bbc_train)

print('\nProcessando dados de teste...\n')

test_words = read_test_data(bbc_test)

Processando dados de treinamento...

	Concluída a leitura de dados para o tópico:  Ad sales boost Time Warner profit


KeyError: 1